# 🚀 EHR-M-GAN Training - Google Colab / VS Code

**Compatible avec:**
- ✅ Google Colab
- ✅ VS Code (Jupyter extension)
- ✅ JupyterLab local

**⚠️ IMPORTANT: Exécutez la CELLULE 1 en premier!**

In [ ]:
# ============================================================
# CELLULE 1: DÉTECTION AUTOMATIQUE DE L'ENVIRONNEMENT
# ============================================================

import os
import sys

print("🔍 Détection de l'environnement...\n")

# Détecter l'environnement
IS_COLAB = 'google.colab' in sys.modules
IS_VSCODE = os.getenv('VSCODE_PID') is not None or os.getenv('TERM_PROGRAM') == 'vscode'

if IS_COLAB:
    print("📱 Environnement: Google Colab")
    ENV = 'COLAB'
elif IS_VSCODE:
    print("💻 Environnement: VS Code")
    ENV = 'VSCODE'
else:
    print("📓 Environnement: Jupyter local")
    ENV = 'LOCAL'

print(f"\n📁 Répertoire actuel: {os.getcwd()}\n")

# ============================================================
# CONFIGURATION SELON L'ENVIRONNEMENT
# ============================================================

if ENV == 'COLAB':
    print("="*70)
    print("🔧 CONFIGURATION GOOGLE COLAB")
    print("="*70)
    
    # Option 1: Fichiers uploadés directement dans Colab
    if os.path.exists('/content/ehrMGAN-main'):
        PROJECT_PATH = '/content/ehrMGAN-main'
        print("✅ Projet trouvé dans /content/ehrMGAN-main")
    
    # Option 2: Fichiers dans Google Drive
    elif os.path.exists('/content/drive/MyDrive/ehrMGAN-main'):
        PROJECT_PATH = '/content/drive/MyDrive/ehrMGAN-main'
        print("✅ Projet trouvé dans Google Drive")
    
    # Option 3: Monter Google Drive
    else:
        print("\n📂 Montage de Google Drive...")
        from google.colab import drive
        drive.mount('/content/drive')
        
        # Chercher dans Drive
        possible_paths = [
            '/content/drive/MyDrive/ehrMGAN-main',
            '/content/drive/My Drive/ehrMGAN-main',
            '/content/ehrMGAN-main'
        ]
        
        PROJECT_PATH = None
        for path in possible_paths:
            if os.path.exists(path):
                PROJECT_PATH = path
                print(f"✅ Projet trouvé: {path}")
                break
        
        if PROJECT_PATH is None:
            print("\n❌ PROJET NON TROUVÉ!")
            print("\n💡 SOLUTIONS:")
            print("\n1️⃣  Cloner depuis GitHub:")
            print("    !git clone https://github.com/VOTRE_REPO/ehrMGAN-main.git /content/ehrMGAN-main")
            print("\n2️⃣  Ou uploader le dossier dans Google Drive à:")
            print("    Mon Drive/ehrMGAN-main/")
            print("\n3️⃣  Ou uploader directement dans Colab (lent):")
            print("    from google.colab import files")
            print("    # Puis uploadez les fichiers un par un")
            raise FileNotFoundError("Projet ehrMGAN-main introuvable")

elif ENV == 'VSCODE' or ENV == 'LOCAL':
    print("="*70)
    print("🔧 CONFIGURATION VS CODE / LOCAL")
    print("="*70)
    
    # Chercher networks.py dans les répertoires proches
    current = os.getcwd()
    search_paths = [
        current,
        os.path.dirname(current),  # Parent
        os.path.dirname(os.path.dirname(current)),  # Grand-parent
    ]
    
    PROJECT_PATH = None
    for path in search_paths:
        if os.path.exists(os.path.join(path, 'networks.py')):
            PROJECT_PATH = path
            print(f"✅ Projet trouvé: {path}")
            break
    
    if PROJECT_PATH is None:
        print("\n❌ PROJET NON TROUVÉ!")
        print(f"\n📁 Répertoire actuel: {current}")
        print("\n💡 SOLUTION:")
        print("\nOuvrez VS Code dans le dossier ehrMGAN-main, puis:")
        print("  File > Open Folder > Sélectionnez ehrMGAN-main")
        print("\nOu utilisez le terminal:")
        print("  cd /chemin/vers/ehrMGAN-main")
        print("  code .")
        raise FileNotFoundError("Projet ehrMGAN-main introuvable")

# ============================================================
# AJOUTER AU PYTHONPATH
# ============================================================

if PROJECT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_PATH)
    print(f"\n✅ Ajouté au PYTHONPATH")

# Naviguer vers le projet
os.chdir(PROJECT_PATH)
print(f"✅ Répertoire de travail changé")

# ============================================================
# VÉRIFICATION DES FICHIERS
# ============================================================

print(f"\n📋 Vérification des fichiers essentiels:")
essential = ['networks.py', 'm3gan.py', 'utils.py', 'Bilateral_lstm_class.py']
all_ok = True

for f in essential:
    exists = os.path.exists(f)
    status = "✅" if exists else "❌"
    print(f"   {status} {f}")
    if not exists:
        all_ok = False

if not all_ok:
    print("\n❌ Fichiers manquants!")
    print("Assurez-vous d'avoir tous les fichiers du projet ehrMGAN-main")
    raise FileNotFoundError("Fichiers du projet manquants")

# ============================================================
# DÉFINIR LES CHEMINS
# ============================================================

DATA_PATH = os.path.join(PROJECT_PATH, "data", "real", "eicu")
CHECKPOINT_DIR = os.path.join(PROJECT_PATH, "data", "checkpoint")
OUTPUT_DIR = os.path.join(PROJECT_PATH, "data", "fake")

# Créer les répertoires
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"\n📂 Chemins configurés:")
print(f"   Projet: {PROJECT_PATH}")
print(f"   Données: {DATA_PATH}")
print(f"   Checkpoints: {CHECKPOINT_DIR}")
print(f"   Output: {OUTPUT_DIR}")

print("\n" + "="*70)
print("✅ CONFIGURATION TERMINÉE!")
print("="*70)
print("\nVous pouvez maintenant exécuter les cellules suivantes.")

In [ ]:
# ============================================================
# CELLULE 2: INSTALLER LES DÉPENDANCES (SI NÉCESSAIRE)
# ============================================================

import sys

print("📦 Vérification des packages...\n")

# Sur Colab, installer les requirements
if ENV == 'COLAB':
    print("📥 Installation des dépendances pour Colab...\n")
    
    # Installer TensorFlow 1.15 (recommandé)
    !pip install -q tensorflow==1.15.0
    
    # Autres dépendances si requirements.txt existe
    if os.path.exists('requirements.txt'):
        !pip install -q -r requirements.txt
    else:
        # Packages essentiels
        !pip install -q numpy pandas matplotlib seaborn scikit-learn
    
    print("✅ Dépendances installées")
    
    # Redémarrer le runtime si TF a été changé
    print("\n⚠️  Si vous venez d'installer TensorFlow 1.15:")
    print("    Runtime > Restart runtime")
    print("    Puis relancez depuis la CELLULE 1")

else:
    print("ℹ️  Environnement local/VS Code")
    print("   Assurez-vous d'avoir installé les dépendances:")
    print("   pip install -r requirements.txt")

In [ ]:
# ============================================================
# CELLULE 3: IMPORT DES PACKAGES
# ============================================================

import tensorflow as tf
import numpy as np
import pickle

print(f"TensorFlow version: {tf.__version__}\n")

# Import des modules du projet
try:
    from networks import C_VAE_NET, D_VAE_NET, C_GAN_NET, D_GAN_NET
    from m3gan import m3gan
    from utils import renormlizer
    print("✅ Tous les modules importés avec succès!")
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    print("\n💡 Retournez à la CELLULE 1 et relancez-la")
    raise

# Gérer TensorFlow 2.x
tf_version = int(tf.__version__.split('.')[0])
if tf_version >= 2:
    print(f"\n⚠️  TensorFlow {tf.__version__} détecté")
    print("   Activation du mode compatibilité TF 1.x...")
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
    print("   ✅ Mode compatibilité activé")
    print("\n   💡 Pour de meilleures performances:")
    print("      pip install tensorflow==1.15")

In [ ]:
# ============================================================
# CELLULE 4: VÉRIFIER ET CHARGER LES DONNÉES
# ============================================================

required_files = [
    'vital_sign_24hrs.pkl',
    'med_interv_24hrs.pkl',
    'statics.pkl'
]

print("📋 Vérification des fichiers de données...\n")

# Vérifier l'existence
all_exist = True
for filename in required_files:
    filepath = os.path.join(DATA_PATH, filename)
    exists = os.path.exists(filepath)
    status = "✅" if exists else "❌"
    print(f"   {status} {filename}")
    if not exists:
        all_exist = False

if not all_exist:
    print(f"\n❌ DONNÉES MANQUANTES!")
    print(f"\nChemin attendu: {DATA_PATH}")
    print("\n💡 SOLUTIONS:")
    
    if ENV == 'COLAB':
        print("\n1️⃣  Uploader les données dans Google Drive:")
        print("    Mon Drive/ehrMGAN-main/data/real/eicu/")
        print("\n2️⃣  Ou uploader directement:")
        print("    from google.colab import files")
        print("    uploaded = files.upload()")
    else:
        print("\n1️⃣  Exécutez d'abord le preprocessing:")
        print("    cd preprocessing_physionet-main/eicu_preprocess")
        print("    python data_extraction_pats.py")
        print("    python data_extraction_post_24hrs_main.py")
    
    raise FileNotFoundError("Fichiers de données manquants")

# Charger les données
print("\n📥 Chargement des données...\n")

with open(os.path.join(DATA_PATH, 'vital_sign_24hrs.pkl'), 'rb') as f:
    vital_labs_3D = pickle.load(f)
    print(f"   ✅ vital_sign_24hrs.pkl - Shape: {vital_labs_3D.shape}")

with open(os.path.join(DATA_PATH, 'med_interv_24hrs.pkl'), 'rb') as f:
    medical_interv_3D = pickle.load(f)
    print(f"   ✅ med_interv_24hrs.pkl - Shape: {medical_interv_3D.shape}")

with open(os.path.join(DATA_PATH, 'statics.pkl'), 'rb') as f:
    statics = pickle.load(f)
    print(f"   ✅ statics.pkl")

continuous_x = vital_labs_3D
discrete_x = medical_interv_3D

print(f"\n✅ Données chargées avec succès!")

In [ ]:
# ============================================================
# CELLULE 5: PARAMÈTRES DES DONNÉES
# ============================================================

time_steps = continuous_x.shape[1]
c_dim = continuous_x.shape[2]
d_dim = discrete_x.shape[2]
no_gen = continuous_x.shape[0]

print(f"📊 Paramètres des données:")
print(f"   Samples: {no_gen}")
print(f"   Time steps: {time_steps}")
print(f"   Continuous features: {c_dim}")
print(f"   Discrete features: {d_dim}")

In [ ]:
# ============================================================
# CELLULE 6: PARAMÈTRES DU MODÈLE
# ============================================================

# Paramètres VAE
c_noise_dim = 5
c_z_size = 100
d_noise_dim = 5
d_z_size = 100

# Construire les réseaux
print("🏗️  Construction des réseaux...\n")

c_vae = C_VAE_NET(x_dim=c_dim, z_dim=c_z_size, time_steps=time_steps)
print("   ✅ C_VAE_NET")

c_gan = C_GAN_NET(x_dim=c_dim, z_dim=c_z_size, time_steps=time_steps)
print("   ✅ C_GAN_NET")

d_vae = D_VAE_NET(x_dim=d_dim, z_dim=d_z_size, time_steps=time_steps)
print("   ✅ D_VAE_NET")

d_gan = D_GAN_NET(x_dim=d_dim, z_dim=d_z_size, time_steps=time_steps)
print("   ✅ D_GAN_NET")

print("\n✅ Réseaux construits!")

In [ ]:
# ============================================================
# CELLULE 7: PARAMÈTRES D'ENTRAÎNEMENT
# ============================================================

# Configuration de base
batch_size = 128
num_pre_epochs = 100  # Réduit pour test rapide sur Colab
num_epochs = 500
epoch_ckpt_freq = 100
epoch_loss_freq = 10

# Rounds
d_rounds = 1
g_rounds = 3
v_rounds = 1

# Learning rates
v_lr_pre = 1e-3
v_lr = 1e-4
g_lr = 1e-4
d_lr = 1e-4

# Coefficients
alpha_re = 1.0
alpha_kl = 1.0
alpha_mt = 1.0
alpha_ct = 1.0
alpha_sm = 1.0

c_beta_adv = 1.0
c_beta_fm = 10.0
d_beta_adv = 1.0
d_beta_fm = 10.0

checkpoint_dir = CHECKPOINT_DIR

print(f"⚙️  Configuration d'entraînement:")
print(f"   Batch size: {batch_size}")
print(f"   Pre-training: {num_pre_epochs} epochs")
print(f"   Training: {num_epochs} epochs")
print(f"   Checkpoints: {checkpoint_dir}")

In [ ]:
# ============================================================
# CELLULE 8: ENTRAÎNEMENT
# ============================================================

print("\n" + "="*70)
print("🚀 DÉBUT DE L'ENTRAÎNEMENT")
print("="*70 + "\n")

if ENV == 'COLAB':
    print("⚠️  Google Colab - Conseils:")
    print("   - Activez le GPU: Runtime > Change runtime type > GPU")
    print("   - L'entraînement prendra plusieurs heures")
    print("   - Les checkpoints seront sauvegardés automatiquement\n")

tf.reset_default_graph()
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth = True

with tf.Session(config=run_config) as sess:
    model = m3gan(
        sess=sess,
        batch_size=batch_size,
        time_steps=time_steps,
        num_pre_epochs=num_pre_epochs,
        num_epochs=num_epochs,
        checkpoint_dir=checkpoint_dir,
        epoch_ckpt_freq=epoch_ckpt_freq,
        epoch_loss_freq=epoch_loss_freq,
        c_dim=c_dim,
        c_noise_dim=c_noise_dim,
        c_z_size=c_z_size,
        c_data_sample=continuous_x,
        c_vae=c_vae,
        c_gan=c_gan,
        d_dim=d_dim,
        d_noise_dim=d_noise_dim,
        d_z_size=d_z_size,
        d_data_sample=discrete_x,
        d_vae=d_vae,
        d_gan=d_gan,
        d_rounds=d_rounds,
        g_rounds=g_rounds,
        v_rounds=v_rounds,
        v_lr_pre=v_lr_pre,
        v_lr=v_lr,
        g_lr=g_lr,
        d_lr=d_lr,
        alpha_re=alpha_re,
        alpha_kl=alpha_kl,
        alpha_mt=alpha_mt,
        alpha_ct=alpha_ct,
        alpha_sm=alpha_sm,
        c_beta_adv=c_beta_adv,
        c_beta_fm=c_beta_fm,
        d_beta_adv=d_beta_adv,
        d_beta_fm=d_beta_fm
    )
    
    model.build()
    model.train()

print("\n" + "="*70)
print("✅ ENTRAÎNEMENT TERMINÉ!")
print("="*70)

In [ ]:
# ============================================================
# CELLULE 9: VÉRIFICATION DES RÉSULTATS
# ============================================================

import glob

print("📊 Vérification des résultats...\n")

synthetic_files = glob.glob(os.path.join(OUTPUT_DIR, "epoch*/gen_data.npz"))
print(f"Fichiers générés: {len(synthetic_files)}")

if synthetic_files:
    print("\n✅ Données synthétiques:")
    for f in sorted(synthetic_files)[-5:]:
        epoch = os.path.basename(os.path.dirname(f))
        size = os.path.getsize(f) / (1024*1024)
        print(f"   {epoch}/gen_data.npz ({size:.2f} MB)")
    
    if ENV == 'COLAB':
        print("\n💾 Pour télécharger les résultats:")
        print("    from google.colab import files")
        print(f"    files.download('{synthetic_files[-1]}')")
else:
    print("⚠️  Aucun fichier généré")

print(f"\n📁 Résultats dans: {OUTPUT_DIR}")